# 6) Presentation Outputs — Governance Intelligence Platform

**Purpose:** generate publication-quality figures and presentation materials from the
backend (Stage 5) demonstrating the platform's capabilities.

**Outputs (aligned to paper figure numbering):**
1. **Fig 1 — System architecture diagram** — pipeline from corpus → KG → RAG backend
2. **Fig 2 — Entity–relationship schema** — the 10 entity types + 11 relation types
3. **Fig 3 — Summary statistics dashboard** — corpus and graph composition
4. **Fig 4 — Governance pathway visualisation** — DfD query example with evidence trail
5. **Fig 5 — Jurisdictional coverage heatmap** — instruments × jurisdictions matrix
6. **Fig 6 — Gap analysis matrix** — practices with barriers vs. enabling instruments
7. **Fig 7 — Demo Q&A cards** — formatted query–answer pairs for slides

**Pipeline:**
- §0 — Dependencies & configuration
- §1 — Load backend state + demo answers
- §2 — Fig 1: System architecture diagram
- §3 — Fig 2: Entity–relationship schema diagram
- §4 — Fig 3: Summary statistics dashboard
- §5 — Fig 4: Governance pathway visualisation
- §6 — Fig 5: Jurisdictional coverage heatmap
- §7 — Fig 6: Gap analysis matrix
- §8 — Fig 7: Demo Q&A cards (formatted)
- §9 — Export all figures


## 0) Install dependencies
Run once per Colab runtime.

In [2]:
!pip -q install anthropic pandas numpy networkx matplotlib seaborn plotly kaleido pillow

import os, json, re, textwrap, pickle
from pathlib import Path
from collections import defaultdict, Counter

import pandas as pd
import numpy as np
import networkx as nx

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
import matplotlib.patheffects as pe

import seaborn as sns

print("✅ Dependencies installed.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.9/405.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
✅ Dependencies installed.


## 1) Configuration + load backend state
Reads the backend artefacts from Stage 5 and the cleaned graph from Stage 4.

In [3]:
# ─────────────────────────────────────────────
# 1a) Configuration
# ─────────────────────────────────────────────
BACKEND_DIR = "/content/drive/MyDrive/ACTIVE/AU_deconstruction_domain/data_analysis/5_backend"
CLEAN_DIR   = "/content/drive/MyDrive/ACTIVE/AU_deconstruction_domain/data_analysis/4_kg_cleaned"
OUTPUT_DIR  = "/content/drive/MyDrive/ACTIVE/AU_deconstruction_domain/data_analysis/6_presentation"

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# ── Clean previous outputs to avoid stale/duplicate files ──
import glob
old_files = glob.glob(os.path.join(OUTPUT_DIR, "fig*")) + glob.glob(os.path.join(OUTPUT_DIR, "manifest.json"))
if old_files:
    for f in old_files:
        os.remove(f)
    print(f"🧹 Cleaned {len(old_files)} previous output files.")

# Colour palette — UoM-inspired professional tones
COLOURS = {
    "Instrument":    "#094183",  # UoM dark blue
    "Authority":     "#0E6EB8",  # medium blue
    "Jurisdiction":  "#5AAFED",  # light blue
    "Requirement":   "#E87722",  # orange
    "Practice":      "#00843D",  # green
    "MaterialAsset": "#8B6914",  # gold/brown
    "Stakeholder":   "#6B2D5B",  # plum
    "Barrier":       "#C4262E",  # red
    "Enabler":       "#2EAF7D",  # teal
    "OutcomeMetric": "#F2C75C",  # amber/yellow
}

ENTITY_ORDER = [
    "Instrument", "Authority", "Jurisdiction", "Requirement",
    "Practice", "MaterialAsset", "Stakeholder",
    "Barrier", "Enabler", "OutcomeMetric",
]

# Figure defaults
plt.rcParams.update({
    "figure.dpi": 200,
    "savefig.dpi": 300,
    "savefig.bbox": "tight",
    "savefig.pad_inches": 0.3,
    "font.family": "sans-serif",
    "font.size": 10,
})

print(f"Config loaded. Output: {OUTPUT_DIR}")


🧹 Cleaned 15 previous output files.
Config loaded. Output: /content/drive/MyDrive/ACTIVE/AU_deconstruction_domain/data_analysis/6_presentation


In [4]:
# ─────────────────────────────────────────────
# 1b) Load backend state + demo answers
# ─────────────────────────────────────────────

# Backend state
with open(os.path.join(BACKEND_DIR, "backend_state.pkl"), "rb") as f:
    state = pickle.load(f)

nodes_df = state["nodes_df"]
edges_df = state["edges_df"]
triples_df = state["triples_df"]
source_chunks = state["source_chunks"]
schema = state["schema"]

# Demo answers
with open(os.path.join(BACKEND_DIR, "demo_answers.json")) as f:
    demos = json.load(f)

# Backend summary
with open(os.path.join(BACKEND_DIR, "backend_summary.json")) as f:
    backend_summary = json.load(f)

# Multi-country schema
with open(os.path.join(CLEAN_DIR, "multi_country_schema.json")) as f:
    mc_schema = json.load(f)

# Post-load dedup (same fix as Notebook 5)
nodes_df["name_norm"] = nodes_df["name_norm"].fillna(nodes_df["name"].str.lower().str.strip())
dupes = nodes_df.groupby(["name_norm", "entity_type"])["node_id"].apply(list)
dupes = dupes[dupes.apply(len) > 1]
dedup_map = {}
for (name_norm, etype), ids in dupes.items():
    for other in ids[1:]:
        dedup_map[other] = ids[0]
if dedup_map:
    edges_df["subject_id"] = edges_df["subject_id"].map(lambda x: dedup_map.get(x, x))
    edges_df["object_id"] = edges_df["object_id"].map(lambda x: dedup_map.get(x, x))
    edges_df = edges_df[edges_df["subject_id"] != edges_df["object_id"]]
    nodes_df = nodes_df[~nodes_df["node_id"].isin(dedup_map.keys())]

# Build graph
G = nx.MultiDiGraph()
for _, row in nodes_df.iterrows():
    G.add_node(row["node_id"], **{k: v for k, v in row.to_dict().items() if pd.notna(v)})
for _, row in edges_df.iterrows():
    attrs = {k: v for k, v in row.to_dict().items() if pd.notna(v) and k not in ("subject_id", "object_id")}
    G.add_edge(row["subject_id"], row["object_id"], **attrs)

print(f"Loaded: {len(nodes_df)} nodes, {len(edges_df)} edges, {len(source_chunks)} chunks")
print(f"Demo answers: {len(demos)} queries")
print(f"Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")


Loaded: 1641 nodes, 3223 edges, 364 chunks
Demo answers: 4 queries
Graph: 1641 nodes, 3223 edges


## 2) System architecture diagram
Pipeline diagram showing: Corpus → Chunking → Three-Pass Extraction → Knowledge Graph → RAG Backend → Query Interface.

Uses pure matplotlib drawing — no external graph libraries needed.

**Paper reference: Figure 1.**


In [5]:
# ─────────────────────────────────────────────
# 2) System architecture diagram
# ─────────────────────────────────────────────

fig, ax = plt.subplots(figsize=(16, 9))
ax.set_xlim(0, 16)
ax.set_ylim(0, 9)
ax.set_aspect("equal")
ax.axis("off")

# Title
ax.text(8, 8.5, "Circular Construction Governance Intelligence Platform",
        ha="center", va="center", fontsize=16, fontweight="bold", color="#094183")
ax.text(8, 8.05, "System Architecture",
        ha="center", va="center", fontsize=12, color="#555555")

# ── Helper functions ──
def draw_box(ax, x, y, w, h, label, sublabel="", colour="#094183", alpha=0.15):
    box = FancyBboxPatch((x, y), w, h, boxstyle="round,pad=0.15",
                          facecolor=colour, alpha=alpha, edgecolor=colour, linewidth=1.5)
    ax.add_patch(box)
    if sublabel:
        ax.text(x + w/2, y + h/2 + 0.15, label, ha="center", va="center",
                fontsize=9, fontweight="bold", color=colour)
        ax.text(x + w/2, y + h/2 - 0.2, sublabel, ha="center", va="center",
                fontsize=7.5, color="#444444", style="italic")
    else:
        ax.text(x + w/2, y + h/2, label, ha="center", va="center",
                fontsize=9, fontweight="bold", color=colour)

def draw_arrow(ax, x1, y1, x2, y2, colour="#094183", label=""):
    ax.annotate("", xy=(x2, y2), xytext=(x1, y1),
                arrowprops=dict(arrowstyle="-|>", color=colour, lw=1.8))
    if label:
        mx, my = (x1+x2)/2, (y1+y2)/2 + 0.18
        ax.text(mx, my, label, ha="center", va="center", fontsize=7, color=colour,
                bbox=dict(boxstyle="round,pad=0.1", facecolor="white",
                          edgecolor="none", alpha=0.9))

# ── Row 1: Data Sources (y ~ 6.3) ──
y1 = 6.3
draw_box(ax, 0.5, y1, 2.5, 1.0, "Policy Corpus",
         f"{len(set(c.get('source_file','') for c in source_chunks))} AU documents\n(laws, standards, guides)", "#094183")
draw_box(ax, 3.8, y1, 2.5, 1.0, "UK Corpus", "Planned", "#AAAAAA", alpha=0.1)
draw_box(ax, 7.1, y1, 2.5, 1.0, "CA Corpus", "Planned", "#AAAAAA", alpha=0.1)

# ── Row 2: Processing Pipeline (y ~ 4.3) ──
y2 = 4.3
draw_box(ax, 0.3, y2, 2.2, 1.0, "PDF Processing",
         f"OCR + semantic\nchunking → {len(source_chunks)} chunks", "#0E6EB8")
draw_box(ax, 3.1, y2, 2.2, 1.0, "Pass 1: Discovery",
         "Claude Haiku\n5,703 mentions", "#00843D")
draw_box(ax, 5.9, y2, 2.2, 1.0, "Pass 2: Consolidate",
         "Embeddings + Sonnet\n596 merges", "#00843D")
draw_box(ax, 8.7, y2, 2.2, 1.0, "Pass 3: Extract",
         "Grounded extraction\nw/ entity registry", "#00843D")
draw_box(ax, 11.5, y2, 2.2, 1.0, "Stage 4: Clean",
         "Cross-type dedup\n+ type correction", "#E87722")

# Arrows: row 1 → row 2
draw_arrow(ax, 1.75, y1, 1.4, y2 + 1.0)
# Arrows: within row 2
for x_start, x_end in [(2.5,3.1), (5.3,5.9), (8.1,8.7), (10.9,11.5)]:
    draw_arrow(ax, x_start, y2+0.5, x_end, y2+0.5)

# ── Row 3: Backend Components (y ~ 2.0) ──
y3 = 2.0
draw_box(ax, 0.5, y3, 3.0, 1.2, "Knowledge Graph",
         f"{G.number_of_nodes():,} nodes, {len(triples_df):,} triples\n10 entity types, 11 relations", "#094183")
draw_box(ax, 4.2, y3, 3.0, 1.2, "Vector Store",
         f"{len(source_chunks)} evidence chunks\nChromaDB + MiniLM-L6", "#6B2D5B")
draw_box(ax, 7.9, y3, 3.0, 1.2, "Query Engine",
         "4 structured query types\n+ free-form RAG", "#00843D")
draw_box(ax, 11.6, y3, 3.0, 1.2, "Claude Sonnet",
         "Evidence-grounded\nnatural language answers", "#E87722")

# Arrow: Stage 4 → KG
draw_arrow(ax, 12.6, y2, 2.0, y3 + 1.2, colour="#888888")

# Arrows: within row 3
draw_arrow(ax, 3.5, y3+0.6, 4.2, y3+0.6, label="evidence\nretrieval")
draw_arrow(ax, 7.2, y3+0.6, 7.9, y3+0.6, label="graph +\nvector")
draw_arrow(ax, 10.9, y3+0.6, 11.6, y3+0.6, label="context +\nquery")

# ── Row 4: Query Types (y ~ 0.3) ──
y4 = 0.3
qw = 3.2
draw_box(ax, 0.3, y4, qw, 0.7, "Governance Pathways", colour="#094183", alpha=0.08)
draw_box(ax, 4.0, y4, qw, 0.7, "Jurisdiction Comparison", colour="#094183", alpha=0.08)
draw_box(ax, 7.7, y4, qw, 0.7, "Gap Analysis", colour="#094183", alpha=0.08)
draw_box(ax, 11.4, y4, qw, 0.7, "Entity Explorer", colour="#094183", alpha=0.08)

# Arrows: query engine → query types
for xc in [1.9, 5.6, 9.3, 13.0]:
    draw_arrow(ax, 9.4, y3, xc, y4 + 0.7, colour="#BBBBBB")

fig.savefig(os.path.join(OUTPUT_DIR, "fig1_architecture.png"))
fig.savefig(os.path.join(OUTPUT_DIR, "fig1_architecture.pdf"))
plt.show()
print("✅ Figure 1 saved: fig1_architecture.png / .pdf")


✅ Figure 1 saved: fig1_architecture.png / .pdf


## 3) Entity–relationship schema diagram
Designed diagram of the 10 entity types with their primary relationships,
showing the governance ontology used across all countries.

**Paper reference: Figure 2.**


In [6]:
# ─────────────────────────────────────────────
# 3) Entity–relationship schema
# ─────────────────────────────────────────────

fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.set_aspect("equal")
ax.axis("off")

ax.text(7, 9.5, "Governance Knowledge Graph Schema",
        ha="center", va="center", fontsize=15, fontweight="bold", color="#094183")
ax.text(7, 9.1, "Entity types and primary relationships",
        ha="center", va="center", fontsize=10, color="#666666")

# Node positions (hand-tuned for readability)
positions = {
    "Authority":     (2.0, 7.5),
    "Instrument":    (6.0, 7.5),
    "Jurisdiction":  (10.5, 7.5),
    "Requirement":   (10.5, 5.5),
    "Practice":      (6.0, 5.0),
    "Enabler":       (2.0, 5.0),
    "Barrier":       (2.0, 2.5),
    "Stakeholder":   (5.0, 2.5),
    "MaterialAsset": (8.5, 2.5),
    "OutcomeMetric": (11.5, 3.5),
}

# Draw entity-type boxes
for etype, (x, y) in positions.items():
    colour = COLOURS[etype]
    count = len(nodes_df[nodes_df["entity_type"] == etype])
    box = FancyBboxPatch((x - 1.2, y - 0.45), 2.4, 0.9,
                          boxstyle="round,pad=0.12",
                          facecolor=colour, alpha=0.18,
                          edgecolor=colour, linewidth=2)
    ax.add_patch(box)
    ax.text(x, y + 0.1, etype, ha="center", va="center",
            fontsize=9, fontweight="bold", color=colour)
    ax.text(x, y - 0.2, f"n = {count}", ha="center", va="center",
            fontsize=7.5, color="#666666")

# Key relationships: (source_type, target_type, label, curvature)
relations = [
    ("Authority",    "Instrument",    "ISSUED_BY",    0),
    ("Instrument",   "Jurisdiction",  "APPLIES_IN",   0),
    ("Instrument",   "Requirement",   "REQUIRES",     0),
    ("Instrument",   "Practice",      "ENABLES",      0.1),
    ("Enabler",      "Practice",      "ENABLES",      0),
    ("Barrier",      "Practice",      "BARRIERS",     0),
    ("Practice",     "OutcomeMetric", "PRODUCES",     0),
    ("Practice",     "MaterialAsset", "INVOLVES",     0),
    ("Stakeholder",  "Practice",      "INVOLVES",     0),
    ("Instrument",   "Practice",      "PROHIBITS",   -0.15),
    ("Requirement",  "Practice",      "APPLIES_TO",   0.15),
]

for src, tgt, label, curve in relations:
    x1, y1 = positions[src]
    x2, y2 = positions[tgt]
    dx, dy = x2 - x1, y2 - y1
    dist = np.sqrt(dx**2 + dy**2)
    if dist > 0:
        ux, uy = dx / dist, dy / dist
        x1s, y1s = x1 + ux * 1.3, y1 + uy * 0.5
        x2s, y2s = x2 - ux * 1.3, y2 - uy * 0.5
    else:
        x1s, y1s, x2s, y2s = x1, y1, x2, y2

    style = f"arc3,rad={curve}" if curve else "arc3,rad=0"
    ax.annotate("", xy=(x2s, y2s), xytext=(x1s, y1s),
                arrowprops=dict(arrowstyle="-|>", color="#888888", lw=1.2,
                                connectionstyle=style))
    mx = (x1s + x2s) / 2 + curve * 1.5
    my = (y1s + y2s) / 2 + 0.15
    ax.text(mx, my, label, ha="center", va="center", fontsize=6.5,
            color="#555555", style="italic",
            bbox=dict(boxstyle="round,pad=0.08", facecolor="white",
                      edgecolor="none", alpha=0.85))

# Country badge
ax.text(12.5, 0.5, "Country: AU", ha="center", va="center", fontsize=8,
        bbox=dict(boxstyle="round,pad=0.3", facecolor="#094183", alpha=0.1,
                  edgecolor="#094183"),
        color="#094183", fontweight="bold")
ax.text(12.5, 0.1, "(UK, CA: same schema)", ha="center", va="center",
        fontsize=7, color="#888888")

fig.savefig(os.path.join(OUTPUT_DIR, "fig2_schema.png"))
fig.savefig(os.path.join(OUTPUT_DIR, "fig2_schema.pdf"))
plt.show()
print("✅ Figure 2 saved: fig2_schema.png / .pdf")


✅ Figure 2 saved: fig2_schema.png / .pdf


## 4) Summary statistics dashboard
Composite 4-panel figure: entity types, relation types, degree distribution, platform metrics.

**Paper reference: Figure 3.**


In [7]:
# ─────────────────────────────────────────────
# 7) Summary statistics dashboard
# ─────────────────────────────────────────────

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle("Governance Intelligence Platform — Summary Statistics",
             fontsize=14, fontweight="bold", color="#094183", y=0.98)

# ── Panel A: Entity type distribution ──
ax = axes[0, 0]
type_counts = nodes_df["entity_type"].value_counts().reindex(ENTITY_ORDER).fillna(0).astype(int)
colours_ordered = [COLOURS[t] for t in ENTITY_ORDER]
bars = ax.barh(ENTITY_ORDER, type_counts.values, color=colours_ordered, alpha=0.7)
ax.set_xlabel("Count")
ax.set_title("(a) Entity Type Distribution", fontsize=11, fontweight="bold")
ax.invert_yaxis()
for bar, count in zip(bars, type_counts.values):
    ax.text(bar.get_width() + 3, bar.get_y() + bar.get_height() / 2,
            str(count), va="center", fontsize=8, color="#444444")

# ── Panel B: Relation type distribution ──
ax = axes[0, 1]
rel_counts = edges_df["predicate"].value_counts()
ax.barh(rel_counts.index, rel_counts.values, color="#094183", alpha=0.5)
ax.set_xlabel("Count")
ax.set_title("(b) Relation Type Distribution", fontsize=11, fontweight="bold")
ax.invert_yaxis()
ax.tick_params(axis="y", labelsize=8)
for i, (idx, count) in enumerate(rel_counts.items()):
    ax.text(count + 3, i, str(count), va="center", fontsize=7, color="#444444")

# ── Panel C: Degree distribution (log-scale) ──
ax = axes[1, 0]
degrees = [G.degree(n) for n in G.nodes()]
ax.hist(degrees, bins=50, color="#094183", alpha=0.6, edgecolor="white")
ax.set_xlabel("Node Degree")
ax.set_ylabel("Frequency")
ax.set_title("(c) Degree Distribution", fontsize=11, fontweight="bold")
ax.set_yscale("log")
mean_deg = np.mean(degrees)
ax.axvline(mean_deg, color="#C4262E", linestyle="--", linewidth=1,
           label=f"Mean = {mean_deg:.1f}")
ax.legend(fontsize=8)

# ── Panel D: Platform summary metrics ──
ax = axes[1, 1]
ax.axis("off")

source_files = set()
for chunk in source_chunks:
    sf = chunk.get("source_file", chunk.get("filename", ""))
    if sf:
        source_files.add(sf)

comps = list(nx.weakly_connected_components(G))
largest_comp = max(comps, key=len)

metrics = [
    ("Source documents",       f"{len(source_files)}"),
    ("Source chunks (kept)",   f"{len(source_chunks)}"),
    ("Entities (nodes)",       f"{G.number_of_nodes():,}"),
    ("Relationships (edges)",  f"{G.number_of_edges():,}"),
    ("Unique triples",         f"{len(triples_df):,}"),
    ("Entity types",           "10"),
    ("Relation types",         "11"),
    ("Graph components",       f"{len(comps)}"),
    ("Largest component",      f"{len(largest_comp):,} ({100*len(largest_comp)/G.number_of_nodes():.0f}%)"),
    ("Countries (current)",    "AU"),
    ("Countries (planned)",    "AU, UK, CA"),
    ("LLM pipeline",           "Haiku → Sonnet → Sonnet"),
    ("Vector store",           f"{len(source_chunks)} chunks (MiniLM-L6)"),
    ("Query types",            "4 structured + free-form"),
]

for i, (label, value) in enumerate(metrics):
    y = 0.95 - i * 0.068
    ax.text(0.05, y, label, fontsize=9, color="#666666",
            transform=ax.transAxes, va="center")
    ax.text(0.65, y, value, fontsize=9, fontweight="bold", color="#094183",
            transform=ax.transAxes, va="center")

ax.set_title("(d) Platform Metrics", fontsize=11, fontweight="bold")

plt.tight_layout(rect=[0, 0, 1, 0.95])
fig.savefig(os.path.join(OUTPUT_DIR, "fig3_dashboard.png"))
fig.savefig(os.path.join(OUTPUT_DIR, "fig3_dashboard.pdf"))
plt.show()
print("✅ Figure 6 saved: fig3_dashboard.png / .pdf")


✅ Figure 6 saved: fig3_dashboard.png / .pdf


## 5) Governance pathway visualisation
A focused subgraph showing the governance pathway for Design for Disassembly (DfD):
instruments, requirements, barriers, enablers, and outcomes connected to DfD.

Capped at 5 neighbours per entity type for readability.

**Paper reference: Figure 4.**


In [8]:
# ─────────────────────────────────────────────
# 4) DfD governance pathway subgraph
# ─────────────────────────────────────────────

# Find DfD node (highest-degree Practice with "disassembly" in name)
dfd_id = None
best_deg = 0
for nid, data in G.nodes(data=True):
    if "disassembly" in data.get("name", "").lower() and data.get("entity_type") == "Practice":
        deg = G.degree(nid)
        if deg > best_deg:
            best_deg = deg
            dfd_id = nid

if not dfd_id:
    print("⚠ DfD node not found — skipping this figure.")
else:
    dfd_name = G.nodes[dfd_id].get("name", "DfD")
    print(f"DfD node: '{dfd_name}' (degree {best_deg})")

    # Collect all 1-hop neighbours with edge info
    neighbours = []
    seen = set()
    for _, tgt, data in G.out_edges(dfd_id, data=True):
        key = (tgt, data.get("predicate", ""))
        if key not in seen:
            seen.add(key)
            neighbours.append({"id": tgt, "pred": data.get("predicate", ""), "dir": "out"})
    for src, _, data in G.in_edges(dfd_id, data=True):
        key = (src, data.get("predicate", ""))
        if key not in seen:
            seen.add(key)
            neighbours.append({"id": src, "pred": data.get("predicate", ""), "dir": "in"})

    # Cap at 5 per entity type, prioritised by degree
    MAX_PER_TYPE = 5
    type_buckets = defaultdict(list)
    for n in neighbours:
        etype = G.nodes[n["id"]].get("entity_type", "Unknown")
        type_buckets[etype].append(n)

    keep = []
    for etype, bucket in type_buckets.items():
        bucket.sort(key=lambda x: -G.degree(x["id"]))
        keep.extend(bucket[:MAX_PER_TYPE])

    # Build filtered subgraph
    sub = nx.DiGraph()
    sub.add_node(dfd_id, **dict(G.nodes[dfd_id]))
    for n in keep:
        sub.add_node(n["id"], **dict(G.nodes[n["id"]]))
        if n["dir"] == "out":
            sub.add_edge(dfd_id, n["id"], predicate=n["pred"])
        else:
            sub.add_edge(n["id"], dfd_id, predicate=n["pred"])

    print(f"Subgraph: {sub.number_of_nodes()} nodes, {sub.number_of_edges()} edges")
    print(f"Types: {Counter(sub.nodes[n].get('entity_type','') for n in sub.nodes if n != dfd_id)}")

    # Layout: use spring with DfD pinned at centre
    pos = nx.spring_layout(sub, seed=42, k=2.8, iterations=100,
                            pos={dfd_id: (0, 0)}, fixed=[dfd_id])

    # Draw
    fig, ax = plt.subplots(figsize=(16, 11))

    # Edges
    for u, v, data in sub.edges(data=True):
        x1, y1 = pos[u]
        x2, y2 = pos[v]
        pred = data.get("predicate", "")
        ax.annotate("", xy=(x2, y2), xytext=(x1, y1),
                    arrowprops=dict(arrowstyle="-|>", color="#CCCCCC", lw=0.8,
                                    shrinkA=15, shrinkB=15))
        mx, my = (x1 + x2) / 2, (y1 + y2) / 2
        ax.text(mx, my + 0.02, pred, fontsize=5, ha="center", va="center",
                color="#999999", style="italic")

    # Nodes
    for nid in sub.nodes():
        x, y = pos[nid]
        nd = sub.nodes[nid]
        etype = nd.get("entity_type", "")
        name = nd.get("name", "")
        colour = COLOURS.get(etype, "#888888")

        if nid == dfd_id:
            ax.scatter(x, y, s=1200, c=colour, alpha=0.25, zorder=2,
                       edgecolors=colour, linewidths=2.5)
            ax.text(x, y, textwrap.fill(name, 22), ha="center", va="center",
                    fontsize=9, fontweight="bold", color=colour, zorder=3)
        else:
            ax.scatter(x, y, s=500, c=colour, alpha=0.18, zorder=2,
                       edgecolors=colour, linewidths=1)
            label = name if len(name) < 35 else name[:32] + "…"
            ax.text(x, y, textwrap.fill(label, 24), ha="center", va="center",
                    fontsize=6.5, color=colour, zorder=3)

    # Legend
    types_present = set(sub.nodes[n].get("entity_type","") for n in sub.nodes())
    legend_handles = [mpatches.Patch(color=COLOURS.get(t,"#888"), alpha=0.5, label=t)
                      for t in ENTITY_ORDER if t in types_present]
    ax.legend(handles=legend_handles, loc="upper left", fontsize=8, framealpha=0.9)

    ax.set_title(f"Governance Pathway: {dfd_name} — 1-Hop Ego Network",
                 fontsize=13, fontweight="bold", color="#094183", pad=15)
    ax.axis("off")

    fig.savefig(os.path.join(OUTPUT_DIR, "fig4_dfd_pathway.png"))
    fig.savefig(os.path.join(OUTPUT_DIR, "fig4_dfd_pathway.pdf"))
    plt.show()
    print(f"✅ Figure 3 saved: fig4_dfd_pathway.png / .pdf ({sub.number_of_nodes()} nodes)")


DfD node: 'Design for Disassembly (DfD)' (degree 133)
Subgraph: 34 nodes, 33 edges
Types: Counter({'Practice': 5, 'Stakeholder': 5, 'Requirement': 4, 'Instrument': 4, 'Enabler': 4, 'OutcomeMetric': 3, 'MaterialAsset': 3, 'Authority': 3, 'Barrier': 2})
✅ Figure 3 saved: fig4_dfd_pathway.png / .pdf (34 nodes)


## 6) Jurisdictional coverage heatmap
Matrix showing which entities connect to which Australian jurisdictions,
based on all edges linking to Jurisdiction-typed nodes.

Since most instruments in the corpus are tagged to "Australia" broadly rather than
to individual states, this figure uses *all entity types* connected to jurisdictions
to give the most informative comparison.

**Paper reference: Figure 5.**


In [9]:
# ─────────────────────────────────────────────
# 5) Jurisdictional coverage heatmap
# ─────────────────────────────────────────────

# Identify jurisdiction nodes
jur_ids = nodes_df[nodes_df["entity_type"] == "Jurisdiction"]["node_id"].tolist()
jur_name_map = {row["node_id"]: row["name"]
                for _, row in nodes_df[nodes_df["entity_type"] == "Jurisdiction"].iterrows()}

# Collect ALL edges involving a jurisdiction node (in either direction)
jur_links = []
for jid in jur_ids:
    jname = jur_name_map[jid]
    # Outgoing from jurisdiction
    for _, tgt, data in G.out_edges(jid, data=True):
        tgt_data = G.nodes[tgt]
        jur_links.append({
            "jurisdiction": jname,
            "entity": tgt_data.get("name", ""),
            "entity_type": tgt_data.get("entity_type", ""),
            "predicate": data.get("predicate", ""),
        })
    # Incoming to jurisdiction
    for src, _, data in G.in_edges(jid, data=True):
        src_data = G.nodes[src]
        jur_links.append({
            "jurisdiction": jname,
            "entity": src_data.get("name", ""),
            "entity_type": src_data.get("entity_type", ""),
            "predicate": data.get("predicate", ""),
        })

jur_df = pd.DataFrame(jur_links).drop_duplicates(subset=["jurisdiction", "entity"])
print(f"Jurisdiction links: {len(jur_df)} unique (jurisdiction, entity) pairs")
print(f"Jurisdictions: {sorted(jur_df['jurisdiction'].unique())}")

# ── Panel A: Count of entity types per jurisdiction ──
fig, axes = plt.subplots(1, 2, figsize=(16, 8), gridspec_kw={"width_ratios": [1.3, 1]})

# Left panel: stacked bar chart
ax = axes[0]
pivot = jur_df.groupby(["jurisdiction", "entity_type"]).size().unstack(fill_value=0)
# Sort jurisdictions by total connections
pivot = pivot.loc[pivot.sum(axis=1).sort_values(ascending=True).index]
# Reorder columns to entity order
cols_present = [c for c in ENTITY_ORDER if c in pivot.columns]
cols_other = [c for c in pivot.columns if c not in cols_present]
pivot = pivot[cols_present + cols_other]

pivot.plot(kind="barh", stacked=True, ax=ax,
           color=[COLOURS.get(c, "#888888") for c in pivot.columns],
           alpha=0.75, edgecolor="white", linewidth=0.5)

ax.set_xlabel("Number of connected entities", fontsize=10)
ax.set_ylabel("")
ax.set_title("(a) Entity Connections by Jurisdiction", fontsize=11, fontweight="bold", color="#094183")
ax.legend(bbox_to_anchor=(0, -0.15), loc="upper left", ncol=3, fontsize=7, framealpha=0.9)
ax.tick_params(axis="y", labelsize=8)

# Add count labels
for container in ax.containers:
    for bar in container:
        w = bar.get_width()
        if w > 2:
            ax.text(bar.get_x() + w / 2, bar.get_y() + bar.get_height() / 2,
                    str(int(w)), ha="center", va="center", fontsize=6, color="white",
                    fontweight="bold")

# ── Panel B: Instrument × Jurisdiction binary heatmap (APPLIES_IN only) ──
ax = axes[1]

applies_in_only = jur_df[jur_df["predicate"] == "APPLIES_IN"].copy()
if len(applies_in_only) > 5:
    # Instrument-type entities only
    inst_links = applies_in_only[applies_in_only["entity_type"] == "Instrument"]
    if len(inst_links) > 0:
        matrix = pd.crosstab(inst_links["entity"], inst_links["jurisdiction"])
        matrix = matrix.loc[matrix.sum(axis=1).sort_values(ascending=False).index]
        matrix = matrix[matrix.sum().sort_values(ascending=False).index]
        TOP = min(25, len(matrix))
        matrix = matrix.head(TOP)

        sns.heatmap(matrix, annot=False, cmap="YlOrRd", linewidths=0.5,
                    linecolor="white", cbar_kws={"label": "Coverage"},
                    ax=ax, vmin=0, vmax=1)
        ax.set_title("(b) Instrument × Jurisdiction", fontsize=11, fontweight="bold", color="#094183")
        ax.tick_params(axis="y", labelsize=6)
        ax.tick_params(axis="x", labelsize=7)
        plt.sca(ax)
        plt.xticks(rotation=45, ha="right")
    else:
        # Fall back: all entity types in APPLIES_IN
        matrix = pd.crosstab(applies_in_only["entity"], applies_in_only["jurisdiction"])
        matrix = matrix.loc[matrix.sum(axis=1).sort_values(ascending=False).index].head(25)
        sns.heatmap(matrix, annot=False, cmap="YlOrRd", linewidths=0.5,
                    linecolor="white", ax=ax, vmin=0, vmax=1)
        ax.set_title("(b) Entities × Jurisdiction (APPLIES_IN)", fontsize=11, fontweight="bold", color="#094183")
        ax.tick_params(axis="y", labelsize=6)
        plt.sca(ax)
        plt.xticks(rotation=45, ha="right")
else:
    # Fall back: use all edges
    matrix = pd.crosstab(jur_df["entity"], jur_df["jurisdiction"])
    matrix = matrix.loc[matrix.sum(axis=1).sort_values(ascending=False).index].head(25)
    sns.heatmap(matrix, annot=False, cmap="YlOrRd", linewidths=0.5,
                linecolor="white", ax=ax, vmin=0, vmax=1)
    ax.set_title("(b) Top Entities × Jurisdiction (all edges)", fontsize=11, fontweight="bold", color="#094183")
    ax.tick_params(axis="y", labelsize=6)
    plt.sca(ax)
    plt.xticks(rotation=45, ha="right")

plt.tight_layout()
fig.savefig(os.path.join(OUTPUT_DIR, "fig5_jurisdiction_heatmap.png"))
fig.savefig(os.path.join(OUTPUT_DIR, "fig5_jurisdiction_heatmap.pdf"))
plt.show()
print(f"✅ Figure 4 saved: fig5_jurisdiction_heatmap.png / .pdf")


Jurisdiction links: 170 unique (jurisdiction, entity) pairs
Jurisdictions: ['AEC industry', 'Australia', 'Australian Cities', 'Australian States and Territories', 'Belgium', 'Brussels', 'Bulli community', 'China', 'Construction Sites', 'Denmark', 'EU', 'Education sector', 'Erith', 'Europe', 'Finland', 'France', 'Germany', 'Government Projects', 'Ireland', 'Japan', 'Leuven', 'Local Government Areas (LGAs)', 'London', 'National Level', 'Netherlands', 'New South Wales (NSW)', 'Northern Territory (NT)', 'Olympic and Paralympic Games', 'Portugal', 'QLD', 'Regional Development', 'Scotland', 'Slovenia', 'Sri Lanka', 'Sweden', 'The Cape', 'United Kingdom (UK)', 'United States (USA)', 'Victoria', 'Western Australia']
✅ Figure 4 saved: fig5_jurisdiction_heatmap.png / .pdf


## 7) Gap analysis matrix
Visualises the governance gaps: practices with barriers but lacking
enabling instruments or policy support. Practices with zero instruments/enablers
are highlighted in red (high-severity gaps).

**Paper reference: Figure 6.**


In [10]:
# ─────────────────────────────────────────────
# 6) Gap analysis matrix
# ─────────────────────────────────────────────

practice_nodes = nodes_df[nodes_df["entity_type"] == "Practice"]

gap_data = []
for _, row in practice_nodes.iterrows():
    pid = row["node_id"]
    name = row["name"]

    barriers = set()
    instruments = set()
    enablers = set()

    # Check both directions for connections
    for src, _, data in G.in_edges(pid, data=True):
        stype = G.nodes[src].get("entity_type", "")
        sname = G.nodes[src].get("name", "")
        if stype == "Barrier":      barriers.add(sname)
        elif stype == "Instrument": instruments.add(sname)
        elif stype == "Enabler":    enablers.add(sname)

    for _, tgt, data in G.out_edges(pid, data=True):
        ttype = G.nodes[tgt].get("entity_type", "")
        tname = G.nodes[tgt].get("name", "")
        if ttype == "Barrier":      barriers.add(tname)
        elif ttype == "Instrument": instruments.add(tname)
        elif ttype == "Enabler":    enablers.add(tname)

    if barriers:
        support = len(instruments) + len(enablers)
        gap_data.append({
            "practice": name,
            "n_barriers": len(barriers),
            "n_instruments": len(instruments),
            "n_enablers": len(enablers),
            "n_support": support,
            "gap_severity": "HIGH" if support == 0 else ("MEDIUM" if len(barriers) > support else "LOW"),
            "gap_ratio": len(barriers) / max(1, support),
        })

gap_df = pd.DataFrame(gap_data).sort_values(
    ["gap_severity", "gap_ratio", "n_barriers"],
    ascending=[True, False, False]
)

TOP_GAPS = min(30, len(gap_df))
plot_df = gap_df.head(TOP_GAPS).copy().reset_index(drop=True)

fig, ax = plt.subplots(figsize=(12, max(7, TOP_GAPS * 0.38)))

y_pos = np.arange(len(plot_df))
bar_h = 0.38

# Barriers (red bars)
ax.barh(y_pos - bar_h/2, plot_df["n_barriers"], height=bar_h,
        color="#C4262E", alpha=0.7, label="Barriers", zorder=2)

# Instruments + Enablers (green bars)
ax.barh(y_pos + bar_h/2, plot_df["n_support"], height=bar_h,
        color="#2EAF7D", alpha=0.7, label="Instruments + Enablers", zorder=2)

ax.set_yticks(y_pos)
ax.set_yticklabels(plot_df["practice"], fontsize=7)
ax.invert_yaxis()
ax.set_xlabel("Count", fontsize=10)
ax.set_title("Governance Gap Analysis: Barriers vs. Policy Support by Practice",
             fontsize=13, fontweight="bold", color="#094183", pad=15)
ax.legend(loc="lower right", fontsize=9, framealpha=0.9)
ax.axvline(x=0, color="black", linewidth=0.5)

# Highlight high-severity gaps
for i, sev in enumerate(plot_df["gap_severity"]):
    if sev == "HIGH":
        ax.get_yticklabels()[i].set_color("#C4262E")
        ax.get_yticklabels()[i].set_fontweight("bold")

# Severity legend
n_high = len(gap_df[gap_df["gap_severity"] == "HIGH"])
n_med = len(gap_df[gap_df["gap_severity"] == "MEDIUM"])
n_low = len(gap_df[gap_df["gap_severity"] == "LOW"])
ax.text(0.98, 0.02,
        f"HIGH severity (0 support): {n_high}\nMEDIUM: {n_med}  |  LOW: {n_low}\n"
        f"Total practices with barriers: {len(gap_df)} / {len(practice_nodes)}",
        transform=ax.transAxes, fontsize=8, va="bottom", ha="right",
        bbox=dict(boxstyle="round,pad=0.4", facecolor="#f8f8f8", edgecolor="#cccccc"))

fig.savefig(os.path.join(OUTPUT_DIR, "fig6_gap_analysis.png"))
fig.savefig(os.path.join(OUTPUT_DIR, "fig6_gap_analysis.pdf"))
plt.show()
print(f"✅ Figure 5 saved: fig6_gap_analysis.png / .pdf ({TOP_GAPS} practices shown)")
print(f"   High-severity gaps: {n_high} | Medium: {n_med} | Low: {n_low}")


✅ Figure 5 saved: fig6_gap_analysis.png / .pdf (30 practices shown)
   High-severity gaps: 29 | Medium: 4 | Low: 19


## 8) Demo Q&A cards (formatted for slides)
Extracts key excerpts from each demo answer and formats them as
presentation-ready text panels — one per query type.

**Paper reference: Figure 7.**


In [11]:
# ─────────────────────────────────────────────
# 8) Demo Q&A cards
# ─────────────────────────────────────────────

card_configs = [
    {
        "key": "demo_1_dfd_pathway",
        "title": "Governance Pathway Query",
        "icon": "Q1",
        "question": "What instruments govern Design for\nDisassembly (DfD) in Australia?",
        "colour": "#094183",
    },
    {
        "key": "demo_2_jurisdiction_comparison",
        "title": "Jurisdictional Comparison",
        "icon": "Q2",
        "question": "How do VIC, NSW, and QLD compare\nin circular economy instruments?",
        "colour": "#0E6EB8",
    },
    {
        "key": "demo_3_gap_analysis",
        "title": "Gap Analysis",
        "icon": "Q3",
        "question": "Which practices face barriers without\nenabling policy support?",
        "colour": "#C4262E",
    },
    {
        "key": "demo_4_free_query",
        "title": "Evidence-Grounded Q&A",
        "icon": "Q4",
        "question": "How do material passports support\nbuilding deconstruction in Australia?",
        "colour": "#00843D",
    },
]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("Query Demonstrations — Governance Intelligence Platform",
             fontsize=14, fontweight="bold", color="#094183", y=0.98)

for ax, config in zip(axes.flat, card_configs):
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.axis("off")
    colour = config["colour"]

    # Card border
    border = FancyBboxPatch((0.2, 0.2), 9.6, 9.6,
                             boxstyle="round,pad=0.3",
                             facecolor="white", edgecolor=colour,
                             linewidth=2, alpha=1)
    ax.add_patch(border)

    # Header bar
    header = FancyBboxPatch((0.3, 8.2), 9.4, 1.4,
                             boxstyle="round,pad=0.15",
                             facecolor=colour, alpha=0.12, edgecolor="none")
    ax.add_patch(header)

    # Icon circle + title
    circle = plt.Circle((1.1, 8.9), 0.35, facecolor=colour, alpha=0.2, edgecolor=colour, linewidth=1.5)
    ax.add_patch(circle)
    ax.text(1.1, 8.9, config["icon"], ha="center", va="center",
            fontsize=10, fontweight="bold", color=colour)
    ax.text(1.8, 9.1, config["title"],
            fontsize=11, fontweight="bold", color=colour)

    # Question
    ax.text(0.8, 8.45, config["question"],
            fontsize=8.5, color="#333333", style="italic")

    # Answer excerpt: extract first meaningful content lines
    answer = demos.get(config["key"], {}).get("answer", "No answer available")
    lines = answer.split("\n")
    excerpt_lines = []
    char_count = 0
    for line in lines:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        line = re.sub(r"\*\*(.+?)\*\*", r"\1", line)  # strip markdown bold
        line = re.sub(r"\*(.+?)\*", r"\1", line)         # strip markdown italic
        if char_count + len(line) > 550:
            remaining = 550 - char_count
            if remaining > 40:
                excerpt_lines.append(line[:remaining] + "…")
            break
        excerpt_lines.append(line)
        char_count += len(line)
        if len(excerpt_lines) >= 7:
            break

    excerpt = "\n".join(excerpt_lines)

    ax.text(0.8, 7.7, "Key Findings:", fontsize=9, fontweight="bold", color=colour)
    ax.text(0.8, 7.2, textwrap.fill(excerpt, 78),
            fontsize=6.8, color="#333333", va="top", linespacing=1.5)

    # Structured data badge at bottom
    result = demos.get(config["key"], {}).get("structured_result", {})
    if isinstance(result, dict):
        counts = result.get("summary_counts", result.get("entity_counts", {}))
        if counts:
            badge_parts = [f"{k}: {v}" for k, v in list(counts.items())[:5]]
            badge_text = "  |  ".join(badge_parts)
            ax.text(0.8, 0.65, badge_text, fontsize=6.5, color="#666666",
                    bbox=dict(boxstyle="round,pad=0.2", facecolor="#f0f0f0",
                              edgecolor="#cccccc"))

plt.tight_layout(rect=[0, 0, 1, 0.95])
fig.savefig(os.path.join(OUTPUT_DIR, "fig7_demo_cards.png"))
fig.savefig(os.path.join(OUTPUT_DIR, "fig7_demo_cards.pdf"))
plt.show()
print("✅ Figure 7 saved: fig7_demo_cards.png / .pdf")


✅ Figure 7 saved: fig7_demo_cards.png / .pdf


## 9) Export all figures + manifest
Creates a manifest of all generated presentation materials with file sizes.


In [12]:
# ─────────────────────────────────────────────
# 9) Export manifest
# ─────────────────────────────────────────────
import glob

all_files = sorted(glob.glob(os.path.join(OUTPUT_DIR, "*")))

manifest = {
    "generated_at": pd.Timestamp.now().isoformat(),
    "output_directory": OUTPUT_DIR,
    "figures": {},
    "summary": {},
}

print(f"{'='*60}")
print(f"STAGE 6 — PRESENTATION OUTPUTS")
print(f"{'='*60}")
print(f"Output directory: {OUTPUT_DIR}\n")

total_size = 0
fig_count = 0
for f in all_files:
    fname = os.path.basename(f)
    size_kb = os.path.getsize(f) / 1024
    total_size += size_kb
    is_fig = fname.startswith("fig")
    marker = "📊" if is_fig else "📄"
    if is_fig:
        fig_count += 1
    print(f"  {marker} {fname:45s} ({size_kb:7.1f} KB)")
    manifest["figures" if is_fig else "summary"][fname] = round(size_kb, 1)

# Save manifest
manifest_path = os.path.join(OUTPUT_DIR, "manifest.json")
with open(manifest_path, "w") as f:
    json.dump(manifest, f, indent=2, default=str)

print(f"\n  Total: {fig_count} figures, {total_size:.0f} KB")
print(f"  Manifest: {manifest_path}")
print(f"\n✅ All presentation materials ready for slides / paper.")


STAGE 6 — PRESENTATION OUTPUTS
Output directory: /content/drive/MyDrive/ACTIVE/AU_deconstruction_domain/data_analysis/6_presentation

  📊 fig1_architecture.pdf                         (   45.6 KB)
  📊 fig1_architecture.png                         (  427.3 KB)
  📊 fig2_schema.pdf                               (   36.4 KB)
  📊 fig2_schema.png                               (  268.3 KB)
  📊 fig3_dashboard.pdf                            (   41.4 KB)
  📊 fig3_dashboard.png                            (  466.2 KB)
  📊 fig4_dfd_pathway.pdf                          (   44.3 KB)
  📊 fig4_dfd_pathway.png                          (  956.2 KB)
  📊 fig5_jurisdiction_heatmap.pdf                 (   50.0 KB)
  📊 fig5_jurisdiction_heatmap.png                 (  693.7 KB)
  📊 fig6_gap_analysis.pdf                         (   32.9 KB)
  📊 fig6_gap_analysis.png                         (  331.8 KB)
  📊 fig7_demo_cards.pdf                           (   48.8 KB)
  📊 fig7_demo_cards.png                        